In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [8]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
    #                                                                 labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
#     ##################################################### repeatable!
#     next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
#                                    reuse=True)
#     next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                                 reuse=True)
#     gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
#                          action_size=action_size, reuse=True)
#     dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
#     gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
#     dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
#                                                                     labels=tf.zeros_like(gQs3))) # 0-1
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
#                                                                      labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [9]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [11]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [12]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [13]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [14]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [15]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [16]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:1.3915 dloss:1.4025 aloss2:1.4267 exploreP:0.9985
Episode:1 meanR:16.0000 R:17.0000 rate:0.0340 aloss:1.3881 dloss:1.3881 aloss2:1.4411 exploreP:0.9968
Episode:2 meanR:15.0000 R:13.0000 rate:0.0260 aloss:1.3674 dloss:1.3812 aloss2:1.4448 exploreP:0.9956
Episode:3 meanR:18.7500 R:30.0000 rate:0.0600 aloss:1.3891 dloss:1.3588 aloss2:1.4688 exploreP:0.9926
Episode:4 meanR:20.0000 R:25.0000 rate:0.0500 aloss:1.3801 dloss:1.3535 aloss2:1.4685 exploreP:0.9901
Episode:5 meanR:18.8333 R:13.0000 rate:0.0260 aloss:1.3820 dloss:1.3335 aloss2:1.4906 exploreP:0.9889
Episode:6 meanR:17.7143 R:11.0000 rate:0.0220 aloss:1.3861 dloss:1.3299 aloss2:1.4929 exploreP:0.9878
Episode:7 meanR:18.0000 R:20.0000 rate:0.0400 aloss:1.3779 dloss:1.3291 aloss2:1.4913 exploreP:0.9858
Episode:8 meanR:17.4444 R:13.0000 rate:0.0260 aloss:1.3794 dloss:1.3084 aloss2:1.5148 exploreP:0.9846
Episode:9 meanR:18.5000 R:28.0000 rate:0.0560 aloss:1.3802 dloss:1.3060 aloss2:1.5

Episode:80 meanR:25.4444 R:25.0000 rate:0.0500 aloss:1.3864 dloss:0.5444 aloss2:3.9876 exploreP:0.8156
Episode:81 meanR:25.4756 R:28.0000 rate:0.0560 aloss:1.3894 dloss:0.5582 aloss2:3.9686 exploreP:0.8134
Episode:82 meanR:25.4699 R:25.0000 rate:0.0500 aloss:1.4061 dloss:0.5839 aloss2:3.8954 exploreP:0.8114
Episode:83 meanR:25.4524 R:24.0000 rate:0.0480 aloss:1.3836 dloss:0.5796 aloss2:4.2632 exploreP:0.8094
Episode:84 meanR:25.7882 R:54.0000 rate:0.1080 aloss:1.3879 dloss:0.5657 aloss2:4.3067 exploreP:0.8051
Episode:85 meanR:25.6977 R:18.0000 rate:0.0360 aloss:1.4104 dloss:0.5468 aloss2:4.6608 exploreP:0.8037
Episode:86 meanR:25.7701 R:32.0000 rate:0.0640 aloss:1.4137 dloss:0.5144 aloss2:4.3068 exploreP:0.8012
Episode:87 meanR:25.9773 R:44.0000 rate:0.0880 aloss:1.4173 dloss:0.5582 aloss2:4.6495 exploreP:0.7977
Episode:88 meanR:25.8315 R:13.0000 rate:0.0260 aloss:1.4030 dloss:0.5454 aloss2:4.1209 exploreP:0.7967
Episode:89 meanR:25.8556 R:28.0000 rate:0.0560 aloss:1.4142 dloss:0.5251 

Episode:159 meanR:38.1400 R:48.0000 rate:0.0960 aloss:1.3940 dloss:0.4628 aloss2:5.2528 exploreP:0.6010
Episode:160 meanR:38.3500 R:39.0000 rate:0.0780 aloss:1.3895 dloss:0.5006 aloss2:5.3597 exploreP:0.5987
Episode:161 meanR:38.1100 R:19.0000 rate:0.0380 aloss:1.3927 dloss:0.4843 aloss2:5.3339 exploreP:0.5976
Episode:162 meanR:38.0300 R:13.0000 rate:0.0260 aloss:1.4155 dloss:0.5243 aloss2:5.4964 exploreP:0.5968
Episode:163 meanR:38.2600 R:40.0000 rate:0.0800 aloss:1.4151 dloss:0.4677 aloss2:5.2074 exploreP:0.5945
Episode:164 meanR:38.3300 R:32.0000 rate:0.0640 aloss:1.4124 dloss:0.5362 aloss2:5.4046 exploreP:0.5926
Episode:165 meanR:38.3200 R:19.0000 rate:0.0380 aloss:1.4051 dloss:0.4843 aloss2:5.4383 exploreP:0.5915
Episode:166 meanR:38.1300 R:22.0000 rate:0.0440 aloss:1.3964 dloss:0.4639 aloss2:5.3232 exploreP:0.5902
Episode:167 meanR:36.5700 R:19.0000 rate:0.0380 aloss:1.3827 dloss:0.5293 aloss2:5.2593 exploreP:0.5891
Episode:168 meanR:36.8400 R:56.0000 rate:0.1120 aloss:1.4058 dlo

Episode:238 meanR:111.2900 R:230.0000 rate:0.4600 aloss:1.4576 dloss:0.5909 aloss2:4.9915 exploreP:0.2300
Episode:239 meanR:115.9400 R:500.0000 rate:1.0000 aloss:1.4490 dloss:0.5943 aloss2:4.9074 exploreP:0.2192
Episode:240 meanR:120.4600 R:497.0000 rate:0.9940 aloss:1.4630 dloss:0.6146 aloss2:4.7713 exploreP:0.2091
Episode:241 meanR:120.7300 R:86.0000 rate:0.1720 aloss:1.4740 dloss:0.6001 aloss2:4.8323 exploreP:0.2074
Episode:242 meanR:124.3500 R:475.0000 rate:0.9500 aloss:1.4436 dloss:0.6116 aloss2:4.8400 exploreP:0.1982
Episode:243 meanR:129.1000 R:500.0000 rate:1.0000 aloss:1.4374 dloss:0.6557 aloss2:4.6456 exploreP:0.1891
Episode:244 meanR:131.3300 R:255.0000 rate:0.5100 aloss:1.4479 dloss:0.6588 aloss2:4.6364 exploreP:0.1845
Episode:245 meanR:133.8200 R:290.0000 rate:0.5800 aloss:1.4475 dloss:0.6308 aloss2:4.6541 exploreP:0.1796
Episode:246 meanR:136.2200 R:351.0000 rate:0.7020 aloss:1.4490 dloss:0.6176 aloss2:4.6354 exploreP:0.1737
Episode:247 meanR:139.0300 R:306.0000 rate:0.61

Episode:316 meanR:228.1100 R:270.0000 rate:0.5400 aloss:1.4401 dloss:0.8721 aloss2:3.8864 exploreP:0.0459
Episode:317 meanR:227.1400 R:115.0000 rate:0.2300 aloss:1.4602 dloss:0.8019 aloss2:3.8267 exploreP:0.0455
Episode:318 meanR:225.3400 R:114.0000 rate:0.2280 aloss:1.4329 dloss:0.7874 aloss2:3.9088 exploreP:0.0451
Episode:319 meanR:225.4500 R:121.0000 rate:0.2420 aloss:1.4193 dloss:0.9116 aloss2:3.8725 exploreP:0.0447
Episode:320 meanR:226.1900 R:122.0000 rate:0.2440 aloss:1.4496 dloss:0.7305 aloss2:3.8745 exploreP:0.0443
Episode:321 meanR:225.8600 R:117.0000 rate:0.2340 aloss:1.4144 dloss:0.8533 aloss2:3.9084 exploreP:0.0439
Episode:322 meanR:226.6700 R:136.0000 rate:0.2720 aloss:1.4268 dloss:0.8282 aloss2:3.8622 exploreP:0.0434
Episode:323 meanR:228.8500 R:370.0000 rate:0.7400 aloss:1.4443 dloss:0.8032 aloss2:3.8555 exploreP:0.0422
Episode:324 meanR:229.0500 R:121.0000 rate:0.2420 aloss:1.4166 dloss:0.8578 aloss2:3.8694 exploreP:0.0418
Episode:325 meanR:230.7700 R:352.0000 rate:0.7

...

Episode:862 meanR:249.2100 R:198.0000 rate:0.3960 aloss:1.0121 dloss:0.9629 aloss2:3.8644 exploreP:0.0100
Episode:863 meanR:249.1100 R:280.0000 rate:0.5600 aloss:1.0135 dloss:0.9692 aloss2:3.8953 exploreP:0.0100
Episode:864 meanR:249.0700 R:286.0000 rate:0.5720 aloss:1.0104 dloss:0.9676 aloss2:3.9143 exploreP:0.0100
Episode:865 meanR:248.3200 R:166.0000 rate:0.3320 aloss:1.0105 dloss:0.9651 aloss2:3.9458 exploreP:0.0100
Episode:866 meanR:247.3900 R:254.0000 rate:0.5080 aloss:1.0115 dloss:0.9647 aloss2:3.9356 exploreP:0.0100
Episode:867 meanR:247.6100 R:276.0000 rate:0.5520 aloss:1.0101 dloss:0.9640 aloss2:3.9546 exploreP:0.0100
Episode:868 meanR:247.2900 R:270.0000 rate:0.5400 aloss:1.0084 dloss:0.9594 aloss2:4.0231 exploreP:0.0100
Episode:869 meanR:246.5600 R:198.0000 rate:0.3960 aloss:1.0098 dloss:0.9611 aloss2:3.9787 exploreP:0.0100
Episode:870 meanR:245.6300 R:187.0000 rate:0.3740 aloss:1.0085 dloss:0.9559 aloss2:4.0764 exploreP:0.0100
Episode:871 meanR:245.1400 R:245.0000 rate:0.4

Episode:940 meanR:257.8900 R:257.0000 rate:0.5140 aloss:0.9948 dloss:0.8905 aloss2:4.9977 exploreP:0.0100
Episode:941 meanR:257.6900 R:241.0000 rate:0.4820 aloss:0.9945 dloss:0.8883 aloss2:5.0452 exploreP:0.0100
Episode:942 meanR:258.8500 R:257.0000 rate:0.5140 aloss:0.9989 dloss:0.8881 aloss2:5.1239 exploreP:0.0100
Episode:943 meanR:258.7800 R:272.0000 rate:0.5440 aloss:0.9944 dloss:0.8885 aloss2:5.0391 exploreP:0.0100
Episode:944 meanR:258.5100 R:308.0000 rate:0.6160 aloss:0.9955 dloss:0.8882 aloss2:5.0977 exploreP:0.0100
Episode:945 meanR:258.5100 R:264.0000 rate:0.5280 aloss:0.9951 dloss:0.8861 aloss2:5.1730 exploreP:0.0100
Episode:946 meanR:259.6000 R:376.0000 rate:0.7520 aloss:0.9951 dloss:0.8867 aloss2:5.0452 exploreP:0.0100
Episode:947 meanR:259.6800 R:269.0000 rate:0.5380 aloss:0.9949 dloss:0.8881 aloss2:5.0698 exploreP:0.0100
Episode:948 meanR:259.6500 R:256.0000 rate:0.5120 aloss:0.9938 dloss:0.8867 aloss2:5.1346 exploreP:0.0100
Episode:949 meanR:260.6300 R:233.0000 rate:0.4

Episode:1018 meanR:291.0600 R:254.0000 rate:0.5080 aloss:0.9953 dloss:0.8917 aloss2:5.4235 exploreP:0.0100
Episode:1019 meanR:292.2200 R:369.0000 rate:0.7380 aloss:0.9947 dloss:0.8858 aloss2:5.4898 exploreP:0.0100
Episode:1020 meanR:293.9200 R:300.0000 rate:0.6000 aloss:0.9961 dloss:0.8891 aloss2:5.3489 exploreP:0.0100
Episode:1021 meanR:294.4300 R:311.0000 rate:0.6220 aloss:0.9953 dloss:0.8856 aloss2:5.3863 exploreP:0.0100
Episode:1022 meanR:295.2100 R:353.0000 rate:0.7060 aloss:0.9950 dloss:0.8868 aloss2:5.3790 exploreP:0.0100
Episode:1023 meanR:295.3100 R:296.0000 rate:0.5920 aloss:0.9957 dloss:0.8867 aloss2:5.4791 exploreP:0.0100
Episode:1024 meanR:295.2200 R:350.0000 rate:0.7000 aloss:0.9931 dloss:0.8841 aloss2:5.4359 exploreP:0.0100
Episode:1025 meanR:295.9500 R:339.0000 rate:0.6780 aloss:0.9942 dloss:0.8804 aloss2:5.4850 exploreP:0.0100
Episode:1026 meanR:297.1500 R:383.0000 rate:0.7660 aloss:0.9934 dloss:0.8767 aloss2:5.6269 exploreP:0.0100
Episode:1027 meanR:296.8000 R:267.000

Episode:1095 meanR:314.3700 R:332.0000 rate:0.6640 aloss:0.9695 dloss:0.7646 aloss2:7.1550 exploreP:0.0100
Episode:1096 meanR:314.3000 R:287.0000 rate:0.5740 aloss:0.9716 dloss:0.7646 aloss2:7.1050 exploreP:0.0100
Episode:1097 meanR:314.7100 R:340.0000 rate:0.6800 aloss:0.9729 dloss:0.7658 aloss2:7.2161 exploreP:0.0100
Episode:1098 meanR:314.3400 R:256.0000 rate:0.5120 aloss:0.9693 dloss:0.7599 aloss2:7.2301 exploreP:0.0100
Episode:1099 meanR:313.8700 R:313.0000 rate:0.6260 aloss:0.9669 dloss:0.7569 aloss2:7.2252 exploreP:0.0100
Episode:1100 meanR:314.0300 R:275.0000 rate:0.5500 aloss:0.9698 dloss:0.7587 aloss2:7.2413 exploreP:0.0100
Episode:1101 meanR:312.9400 R:271.0000 rate:0.5420 aloss:0.9705 dloss:0.7611 aloss2:7.1733 exploreP:0.0100
Episode:1102 meanR:312.3300 R:296.0000 rate:0.5920 aloss:0.9693 dloss:0.7603 aloss2:7.3453 exploreP:0.0100
Episode:1103 meanR:312.3300 R:254.0000 rate:0.5080 aloss:0.9705 dloss:0.7636 aloss2:7.1618 exploreP:0.0100
Episode:1104 meanR:312.8300 R:322.000

Episode:1172 meanR:311.3900 R:334.0000 rate:0.6680 aloss:0.9630 dloss:0.7319 aloss2:8.1582 exploreP:0.0100
Episode:1173 meanR:310.0300 R:266.0000 rate:0.5320 aloss:0.9646 dloss:0.7313 aloss2:8.1835 exploreP:0.0100
Episode:1174 meanR:309.5100 R:274.0000 rate:0.5480 aloss:0.9645 dloss:0.7344 aloss2:8.3279 exploreP:0.0100
Episode:1175 meanR:310.7000 R:382.0000 rate:0.7640 aloss:0.9633 dloss:0.7270 aloss2:8.2965 exploreP:0.0100
Episode:1176 meanR:310.7500 R:371.0000 rate:0.7420 aloss:0.9647 dloss:0.7256 aloss2:8.1599 exploreP:0.0100
Episode:1177 meanR:309.2300 R:263.0000 rate:0.5260 aloss:0.9643 dloss:0.7305 aloss2:8.2854 exploreP:0.0100
Episode:1178 meanR:310.1100 R:377.0000 rate:0.7540 aloss:0.9614 dloss:0.7193 aloss2:8.2889 exploreP:0.0100
Episode:1179 meanR:308.9300 R:245.0000 rate:0.4900 aloss:0.9650 dloss:0.7257 aloss2:8.4016 exploreP:0.0100
Episode:1180 meanR:307.9000 R:244.0000 rate:0.4880 aloss:0.9638 dloss:0.7260 aloss2:8.5095 exploreP:0.0100
Episode:1181 meanR:307.8800 R:271.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.